In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np

In [2]:
# load our dataset
import vpower
snapshot = '/appalachia/d5/DISK/from_pleiades/snapshots/gmcs0_wind0_gmc9/snapshot_550.hdf5'
simPtcls = vpower.SimulationParticles.load_snapshot(snapshot)
data = simPtcls.pos
query = vpower.make_grid_coords(Lbox=1., Nsize=256)

Only the CPU-only flavour of Voxelize is available!


## nmslib
Task: create tree and perform nearest neighbor search for 256**3 points.
|      | hnsw | sw-graph | vptree | napp | simple_invindx | brute_force |
| ---- | :--: | :------: | :----: | :--: | :------------: | :---------: |
| time | 115s | 181s | 96.6s | > 1 hour | doesn't work in l2 space | > 1 hour |



In [4]:
import nmslib
import time

for method in ['brute_force']:
    t0 = time.perf_counter()
    # initialize a new index, using a HNSW index on Cosine Similarity
    index = nmslib.init(method=method, space='l2')
    index.addDataPointBatch(data)
    print("Batch added.")

    index.createIndex(print_progress=False)
    print("Index created.")
    # query for the nearest neighbours of the first datapoint
    # ids, distances = index.knnQuery(query[0], k=1)

    # get all nearest neighbours for all the datapoint
    # using a pool of 4 threads to compute
    neighbours = index.knnQueryBatch(query, k=1, num_threads=20) # put query data here

    print("Time taken", time.perf_counter() - t0)

Batch added.
Index created.


## NGT

In [5]:
import ngtpy
# import random
# dim = 10
# nb = 100
# vectors = [[random.random() for _ in range(dim)] for _ in range(nb)]
# query = vectors[0]

ngtpy.create(b"tmp", 3)
index = ngtpy.Index(b"tmp")
index.batch_insert(data)

results = index.search(query[0,:], 1)
for i, (id, distance) in enumerate(results) :
    print(str(i) + ": " + str(id) + ", " + str(distance))
    object = index.get_object(id)
    print(object)

0: 7505772, 0.03123338893055916
[0.01290084794163704, 0.0296783410012722, 0.011279667727649212]


In [4]:
np.shape(query)

(16777216, 3)